# 
# JSON structured data from tables in scientific papers

## Jsonformer non-token approach for OpenAI
#### (based on: https://github.com/1rgs/jsonformer, https://github.com/martinezpl/jsonformer/tree/add-openai)

In [1]:
import os
import pandas as pd
import json
import time

import openai
from IPython.display import IFrame
from IPython.display import HTML

from jsonformer_non_tokens import JsonformerNoTokens, OpenAIModel
from utils import highlight_values, get_driver, extract_tableSource, extract_table, quit_driver

DATA_DIR = "your_path"
chromedriver_path = os.path.join(DATA_DIR, "chromedriver.exe")

openai.api_key = "your_openai_api_key"

## Example: Biomass properties

### Get table HTML code from DOI

In [61]:
doi = "10.1016/j.enconman.2018.09.020"

In [62]:
IFrame(src="https://dx.doi.org/" + doi, width=900, height=380)

In [63]:
driver = get_driver(doi, chromedriver_path)
time.sleep(2)
tableSource = extract_tableSource(driver)

[<selenium.webdriver.remote.webelement.WebElement (session="c63dda3d936e3c83c988889b71294c83", element="83E91D3E8DDD28B9807B3219B64CE34F_element_73")>]
number of tables = 1


In [64]:
table_num = 1
html_table_biomass = extract_table(tableSource, table_num)
quit_driver(driver)

In [65]:
html_table_biomass

'<table><thead><tr class="valign-top"><th scope="col" class="align-left rowsep-1" rowspan="3">Sample</th><th scope="col" class="align-left" colspan="5">Ultimate analysis</th><th scope="col" class="align-left rowsep-1" colspan="4">Proximate analysis</th><th scope="col" class="align-left">HHV</th><th scope="col" class="align-left">H/O</th><th scope="col" class="align-left">He density</th></tr><tr class="valign-top"><th scope="col" class="align-left rowsep-1" colspan="5">(wt%, db)</th><th scope="col" class="align-left">(wt%)</th><th scope="col" class="align-left rowsep-1" colspan="3">(wt%, db)</th><th scope="col" class="align-left">(MJ/kg, db)</th><th scope="col" class="align-left">ratio</th><th scope="col" class="align-left">(g/cm<sup>3</sup>)<a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#tblfn2" name="btblfn2"><span class="anchor-text"><sup>b</sup></span></a></th></tr><tr class="rowsep-1 valign-top"><th scope="col" class="align-left">C</th><th scope="col" c

In [66]:
with open("html_table_biomass.txt", "w") as file:
    file.write(html_table_biomass)

### Show table

In [68]:
HTML(html_table_biomass)

### Convert HTML table to JSON format with openAI LLMs

In [69]:
json_schema_biomass = {
    "type": "object",
    "properties": {
        "Biomass type": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "Sample": {"type": "string"},
                    "Ultimate Analysis (wt%, db)": {
                        "type": "object",
                        "properties": {
                            "C": {"type": "number"},
                            "N": {"type": "number"},
                            "H": {"type": "number"},
                            "S": {"type": "number"},
                            "O": {"type": "number"},
                        },
                    },
                    "Proximate Analysis (wt%, db)": {
                        "type": "object",
                        "properties": {
                            "MC (wt%)": {"type": "number"},
                            "Ash": {"type": "number"},
                            "VM": {"type": "number"},
                            "FC": {"type": "number"},
                        },
                    },
                    "HHV (MJ/kg, db)": {"type": "number"},
                    "H/O": {"type": "number"},
                    "He density (g/cm3)": {"type": "number"}, 
                },
            },
        },
    },
}

In [70]:
with open('json_schema_biomass.json', 'w') as file:
    json.dump(json_schema_biomass, file)
with open("json_schema_biomass.json") as file:
    json_schema_biomass = json.load(file)
    file.close()

In [71]:
json_schema_biomass

{'type': 'object',
 'properties': {'Biomass type': {'type': 'array',
   'items': {'type': 'object',
    'properties': {'Sample': {'type': 'string'},
     'Ultimate Analysis (wt%, db)': {'type': 'object',
      'properties': {'C': {'type': 'number'},
       'N': {'type': 'number'},
       'H': {'type': 'number'},
       'S': {'type': 'number'},
       'O': {'type': 'number'}}},
     'Proximate Analysis (wt%, db)': {'type': 'object',
      'properties': {'MC (wt%)': {'type': 'number'},
       'Ash': {'type': 'number'},
       'VM': {'type': 'number'},
       'FC': {'type': 'number'}}},
     'HHV (MJ/kg, db)': {'type': 'number'},
     'H/O': {'type': 'number'},
     'He density (g/cm3)': {'type': 'number'}}}}}}

In [72]:
builder = JsonformerNoTokens(
    model=OpenAIModel("text-davinci-003", debug=False),
    json_schema=json_schema_biomass,
    text=html_table_biomass,
    prompt="Generate a object with the following schema extracting the information from the provided table in html code:",
    temperature=0.1,
    debug=True,
    max_array_length=10,
    max_string_token_length=10,
)

print("Generating...")
result_biomass = builder()

Generating...
[generate_string] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table><thead><tr class="valign-top"><th scope="col" class="align-left rowsep-1" rowspan="3">Sample</th><th scope="col" class="align-left" colspan="5">Ultimate analysis</th><th scope="col" class="align-left rowsep-1" colspan="4">Proximate analysis</th><th scope="col" class="align-left">HHV</th><th scope="col" class="align-left">H/O</th><th scope="col" class="align-left">He density</th></tr><tr class="valign-top"><th scope="col" class="align-left rowsep-1" colspan="5">(wt%, db)</th><th scope="col" class="align-left">(wt%)</th><th scope="col" class="align-left rowsep-1" colspan="3">(wt%, db)</th><th scope="col" class="align-left">(MJ/kg, db)</th><th scope="col" class="align-left">ratio</th><th scope="col" class="align-left">(g/cm<sup>3</sup>)<a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#tblfn2" name="btblfn2"><s

[generate_number] response  49.44, "N
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table><thead><tr class="valign-top"><th scope="col" class="align-left rowsep-1" rowspan="3">Sample</th><th scope="col" class="align-left" colspan="5">Ultimate analysis</th><th scope="col" class="align-left rowsep-1" colspan="4">Proximate analysis</th><th scope="col" class="align-left">HHV</th><th scope="col" class="align-left">H/O</th><th scope="col" class="align-left">He density</th></tr><tr class="valign-top"><th scope="col" class="align-left rowsep-1" colspan="5">(wt%, db)</th><th scope="col" class="align-left">(wt%)</th><th scope="col" class="align-left rowsep-1" colspan="3">(wt%, db)</th><th scope="col" class="align-left">(MJ/kg, db)</th><th scope="col" class="align-left">ratio</th><th scope="col" class="align-left">(g/cm<sup>3</sup>)<a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#t

[generate_number] response  5.85, "S
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table><thead><tr class="valign-top"><th scope="col" class="align-left rowsep-1" rowspan="3">Sample</th><th scope="col" class="align-left" colspan="5">Ultimate analysis</th><th scope="col" class="align-left rowsep-1" colspan="4">Proximate analysis</th><th scope="col" class="align-left">HHV</th><th scope="col" class="align-left">H/O</th><th scope="col" class="align-left">He density</th></tr><tr class="valign-top"><th scope="col" class="align-left rowsep-1" colspan="5">(wt%, db)</th><th scope="col" class="align-left">(wt%)</th><th scope="col" class="align-left rowsep-1" colspan="3">(wt%, db)</th><th scope="col" class="align-left">(MJ/kg, db)</th><th scope="col" class="align-left">ratio</th><th scope="col" class="align-left">(g/cm<sup>3</sup>)<a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#tb

[generate_number] response  42.90}, "Pro
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table><thead><tr class="valign-top"><th scope="col" class="align-left rowsep-1" rowspan="3">Sample</th><th scope="col" class="align-left" colspan="5">Ultimate analysis</th><th scope="col" class="align-left rowsep-1" colspan="4">Proximate analysis</th><th scope="col" class="align-left">HHV</th><th scope="col" class="align-left">H/O</th><th scope="col" class="align-left">He density</th></tr><tr class="valign-top"><th scope="col" class="align-left rowsep-1" colspan="5">(wt%, db)</th><th scope="col" class="align-left">(wt%)</th><th scope="col" class="align-left rowsep-1" colspan="3">(wt%, db)</th><th scope="col" class="align-left">(MJ/kg, db)</th><th scope="col" class="align-left">ratio</th><th scope="col" class="align-left">(g/cm<sup>3</sup>)<a class="anchor workspace-trigger u-display-inline anchor-paragraph" href=

[generate_number] response  1.45, "VM
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table><thead><tr class="valign-top"><th scope="col" class="align-left rowsep-1" rowspan="3">Sample</th><th scope="col" class="align-left" colspan="5">Ultimate analysis</th><th scope="col" class="align-left rowsep-1" colspan="4">Proximate analysis</th><th scope="col" class="align-left">HHV</th><th scope="col" class="align-left">H/O</th><th scope="col" class="align-left">He density</th></tr><tr class="valign-top"><th scope="col" class="align-left rowsep-1" colspan="5">(wt%, db)</th><th scope="col" class="align-left">(wt%)</th><th scope="col" class="align-left rowsep-1" colspan="3">(wt%, db)</th><th scope="col" class="align-left">(MJ/kg, db)</th><th scope="col" class="align-left">ratio</th><th scope="col" class="align-left">(g/cm<sup>3</sup>)<a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#t

...

[generate_number] response  78.4, "FC
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table><thead><tr class="valign-top"><th scope="col" class="align-left rowsep-1" rowspan="3">Sample</th><th scope="col" class="align-left" colspan="5">Ultimate analysis</th><th scope="col" class="align-left rowsep-1" colspan="4">Proximate analysis</th><th scope="col" class="align-left">HHV</th><th scope="col" class="align-left">H/O</th><th scope="col" class="align-left">He density</th></tr><tr class="valign-top"><th scope="col" class="align-left rowsep-1" colspan="5">(wt%, db)</th><th scope="col" class="align-left">(wt%)</th><th scope="col" class="align-left rowsep-1" colspan="3">(wt%, db)</th><th scope="col" class="align-left">(MJ/kg, db)</th><th scope="col" class="align-left">ratio</th><th scope="col" class="align-left">(g/cm<sup>3</sup>)<a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#t

[generate_number] response  19.8}, "HH
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table><thead><tr class="valign-top"><th scope="col" class="align-left rowsep-1" rowspan="3">Sample</th><th scope="col" class="align-left" colspan="5">Ultimate analysis</th><th scope="col" class="align-left rowsep-1" colspan="4">Proximate analysis</th><th scope="col" class="align-left">HHV</th><th scope="col" class="align-left">H/O</th><th scope="col" class="align-left">He density</th></tr><tr class="valign-top"><th scope="col" class="align-left rowsep-1" colspan="5">(wt%, db)</th><th scope="col" class="align-left">(wt%)</th><th scope="col" class="align-left rowsep-1" colspan="3">(wt%, db)</th><th scope="col" class="align-left">(MJ/kg, db)</th><th scope="col" class="align-left">ratio</th><th scope="col" class="align-left">(g/cm<sup>3</sup>)<a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#

[generate_number] response  20.767, "H
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table><thead><tr class="valign-top"><th scope="col" class="align-left rowsep-1" rowspan="3">Sample</th><th scope="col" class="align-left" colspan="5">Ultimate analysis</th><th scope="col" class="align-left rowsep-1" colspan="4">Proximate analysis</th><th scope="col" class="align-left">HHV</th><th scope="col" class="align-left">H/O</th><th scope="col" class="align-left">He density</th></tr><tr class="valign-top"><th scope="col" class="align-left rowsep-1" colspan="5">(wt%, db)</th><th scope="col" class="align-left">(wt%)</th><th scope="col" class="align-left rowsep-1" colspan="3">(wt%, db)</th><th scope="col" class="align-left">(MJ/kg, db)</th><th scope="col" class="align-left">ratio</th><th scope="col" class="align-left">(g/cm<sup>3</sup>)<a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#

[generate_number] response  2.53, "He
[generate_number] prompt Generate a object with the following schema extracting the information from the provided table in html code:
<table><thead><tr class="valign-top"><th scope="col" class="align-left rowsep-1" rowspan="3">Sample</th><th scope="col" class="align-left" colspan="5">Ultimate analysis</th><th scope="col" class="align-left rowsep-1" colspan="4">Proximate analysis</th><th scope="col" class="align-left">HHV</th><th scope="col" class="align-left">H/O</th><th scope="col" class="align-left">He density</th></tr><tr class="valign-top"><th scope="col" class="align-left rowsep-1" colspan="5">(wt%, db)</th><th scope="col" class="align-left">(wt%)</th><th scope="col" class="align-left rowsep-1" colspan="3">(wt%, db)</th><th scope="col" class="align-left">(MJ/kg, db)</th><th scope="col" class="align-left">ratio</th><th scope="col" class="align-left">(g/cm<sup>3</sup>)<a class="anchor workspace-trigger u-display-inline anchor-paragraph" href="#t

[generate_number] response  1.221}]}]}


In [73]:
highlight_values(result_biomass)

{
  Biomass type: [
    {
      Sample: "AS",
      Ultimate Analysis (wt%, db): {
        C: 49.44,
        N: 0.31,
        H: 5.85,
        S: 0.05,
        O: 42.9
      },
      Proximate Analysis (wt%, db): {
        MC (wt%): 6.5,
        Ash: 1.45,
        VM: 78.9,
        FC: 19.6
      },
      HHV (MJ/kg, db): 19.565,
      H/O: 2.16,
      He density (g/cm3): 1.252
    },
    {
      Sample: "CHE",
      Ultimate Analysis (wt%, db): {
        C: 50.22,
        N: 0.34,
        H: 5.55,
        S: 0.01,
        O: 43.41
      },
      Proximate Analysis (wt%, db): {
        MC (wt%): 8.4,
        Ash: 0.47,
        VM: 81.2,
        FC: 18.3
      },
      HHV (MJ/kg, db): 19.109,
      H/O: 2.03,
      He density (g/cm3): 1.268
    },
    {
      Sample: "CHET",
      Ultimate Analysis (wt%, db): {
        C: 51.3,
        N: 0.4,
        H: 5.4,
        S: 0.02,
        O: 42.59
      },
      Proximate Analysis (wt%, db): {
        MC (wt%): 8.2,
        Ash: 0.29,
     

In [74]:
with open('json_table_biomass.json', 'w') as file:
    json.dump(result_biomass, file)